In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split 
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler 
from sklearn.pipeline import Pipeline,make_pipeline 
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier 

In [3]:
titanic_dataset = pd.read_csv('train.csv')
titanic_dataset.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
101,102,0,3,"Petroff, Mr. Pastcho (""Pentcho"")",male,NaN,0,0,349215,7.8958,NaN,S
245,246,0,1,"Minahan, Dr. William Edward",male,44.0,2,0,19928,90.0000,C78,Q
190,191,1,2,"Pinsky, Mrs. (Rosa)",female,32.0,0,0,234604,13.0000,NaN,S
620,621,0,3,"Yasbeck, Mr. Antoni",male,27.0,1,0,2659,14.4542,NaN,C
432,433,1,2,"Louch, Mrs. Charles Alexander (Alice Adelaide ...",female,42.0,1,0,SC/AH 3085,26.0000,NaN,S


In [4]:
# here in this we dorping this columns in the dataframe becues which is not usefull for machine learning pipeline 
titanic_dataset = titanic_dataset.drop(columns =['PassengerId','Name','Ticket','Cabin'],axis = 1) # here it not used for this in our dataframe 

In [5]:
titanic_dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
titanic_dataset.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [7]:
x_train,x_test,y_train,y_test = train_test_split(titanic_dataset.iloc[:,1:],titanic_dataset.iloc[:,-8],test_size = 0.2,random_state= 42)

In [8]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [9]:
y_train.sample(6)

752    0
774    1
373    0
43     1
475    0
27     0
Name: Survived, dtype: int64

In [10]:
# imputation transoform 
trf1 = ColumnTransformer([
  ('impute_age',SimpleImputer(),[2]),   # here [2] is the index for the dataframe in age column index 
  ('impute_embarked',SimpleImputer(strategy = 'most_frequent'),[6])
],remainder = 'passthrough') # here when we do not do the passthrough than all the extra columns will be drop in the data frame

In [11]:
# one hot encoding 
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse = False,handle_unknown = 'ignore'),[1,6])
],remainder = 'passthrough')

In [12]:
# scling 
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])   # here we used the MinMixscale is used thar for the freature selction 

In [13]:
# Feature selection 
trf4 = SelectKBest(score_func = chi2,k=8)
# k = 8 ----> it taking the top 8 feature form the dataframe 

In [14]:
# trian the model
trf5 = DecisionTreeClassifier()

# create Pipeline

In [15]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])
# here in this we parcticler transformer name auka object

# Pipeline Vs make_pipeline 
<br>
Pipeline requires naming of steps,make_pipeline does not.
<br>
<br>
(same applies to ColumnTransformer vs make_column_transformer)

In [16]:
# Alternate syntax 
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [17]:
# train 
pipe.fit(x_train,y_train)

C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001B821263E20>)),
                ('trf5', DecisionTreeClassifier())])

In [18]:
# here this codes shows the steps of pip line 
pipe.named_steps['trf1'].transformers_[0][1].statistics_ # statistics ----> it shows the mean values 

array([29.49884615])

In [19]:
# Display pipeline 

from sklearn import set_config
set_config(display = 'diagram')

In [22]:
# predict 
y_pred = pipe.predict(x_test) # here pipe is machine learning model in the dataframe

In [21]:
y_pred 

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [25]:
from sklearn.metrics import accuracy_score  # here our model accuracy is bad 
accuracy_score(y_test,y_pred)  # y_test ----> it is those data which is showed to predict 

0.6256983240223464

# Cross Validation using Pipeline 

In [29]:
# cross validation using cross_val_score 
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train ,cv = 5,scoring = 'accuracy').mean()

C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `spa

0.6391214419383433

# GridSearch using Pipeline

In [30]:
# gridsearchev
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [31]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv =5,scoring = 'accuracy')
grid.fit(x_train,y_train)

C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `spa

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000001B821263E20>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [32]:
grid.best_score_

0.6391214419383433

In [33]:
grid.best_params_

{'trf5__max_depth': 2}

# Exporting the Pipeline

In [34]:
# export 
import pickle 
pickle.dump(pipe,open('pipe.pkl','wb'))